### PROJECT 1

In [ ]:
# imports
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager

# hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory

# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net
from source.custom_cnn.dropout_batch_norm_cnn import DBN_cnn

In [ ]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
gpus_per_trial = 2
max_num_epochs = 20
num_samples = 10
local_dir = ConfigManager.get_tuning_results_path()

config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD, optim.Adam]),
    "lr": tune.uniform(0.001, 0.01),
    "weight_decay": tune.choice([0, 0.0001, 0.001, 0.01])
}

config_net_1 = {
    "type": tune.choice([DBN_cnn]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "n_blocks": tune.choice([1, 3]),
    "is_batch_norm": tune.choice([True, False]),
    "is_fc_drop_out": tune.choice([True, False]),
    "is_batch_norm_fc": tune.choice([True, False]),
    "is_drop_out": tune.choice([True, False])
}

config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_kaggle())
    ]),
    "batch_size": tune.choice([8, 32, 64]),
    "epochs": 150,
    "net": tune.choice([config_net_1]),
    "tuning_id": "DBN"
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20, overwrite=True, max_report_frequency=10, max_error_rows=5)

In [ ]:
tuner = HyperparameteresTunner()

In [ ]:
best_model = tuner.tune(dataset_name, local_dir, config, scheduler, reporter, num_samples=4)